In [ ]:
# =========================================================
# Fase 1: Extracción de wallets
# 
# En esta fase nos conectamos a la API pública de Plume y
# descargamos las wallets que aparecen en el leaderboard.
# Guardamos los resultados en un archivo JSON para trabajar
# con ellos en las siguientes fases.
# =========================================================

import requests
import json
import os

# === Configuración de la API ===
BASE_URL = "https://portal-api.plume.org/api/v1/stats/leaderboard"
COUNT_PER_PAGE = 2000   # Máximo número de wallets por consulta
TIMEOUT_SECONDS = 30    # Tiempo de espera por request

# === Configuración de archivos ===
OUTPUT_FOLDER = "/home/ismael/Desktop/Diario_de_un_farmer/Plume_network/data"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
OUTPUT_FILE = os.path.join(OUTPUT_FOLDER, "plume_wallets.json")


def obtener_wallets_unicas_y_guardar():
    """
    Descarga todas las wallets del leaderboard de Plume.
    Se detiene cuando ya no hay datos o cuando el XP llega a cero.
    """
    offset = 0
    wallets_unicas = set()

    while True:
        # Parámetros de la consulta
        params = {
            "offset": offset,
            "count": COUNT_PER_PAGE,
            "walletAddress": "undefined",
            "overrideDay1Override": "false",
            "preview": "false",
        }

        # Llamada a la API
        respuesta = requests.get(BASE_URL, params=params, timeout=TIMEOUT_SECONDS)
        respuesta.raise_for_status()
        datos_pagina = respuesta.json().get("data", {}).get("leaderboard", [])

        # Si no hay más datos → salir del loop
        if not datos_pagina:
            break

        # Procesamos cada wallet de la página
        for entrada in datos_pagina:
            wallet = entrada.get("walletAddress")
            xp = entrada.get("totalXp", 0)

            # Si encontramos XP=0 → detenemos la extracción
            if xp == 0:
                datos_pagina = []
                break

            wallets_unicas.add(wallet)

        offset += COUNT_PER_PAGE
        if not datos_pagina:  # condición de salida
            break

    # === Guardar resultados en JSON ===
    wallets_lista = [{"walletAddress": w} for w in wallets_unicas]

    with open(OUTPUT_FILE, "w") as f:
        json.dump(wallets_lista, f, indent=2)

    print(f"Guardadas {len(wallets_unicas):,} wallets en {OUTPUT_FILE}")
    return len(wallets_unicas)


# === Ejecutar extracción ===
if __name__ == "__main__":
    cantidad_wallets = obtener_wallets_unicas_y_guardar()
    print(f"🔹 Total wallets únicas en leaderboard: {cantidad_wallets:,}")


In [ ]:
# =========================================================
# Fase 2: Procesamiento y Enriquecimiento de wallets
#
# En esta fase consultamos dos APIs:
#  1. Portal de Plume → estadísticas por wallet.
#  2. Explorer → transacciones de cada wallet.
#
# Con esta información obtenemos:
#   - Total XP
#   - Referidos y referido por
#   - Puntos por Plume Guardians
#   - Actividad (transacciones y días activos)
#   - Flag sybil (clasificación básica)
#
# Los resultados se guardan en un JSON enriquecido.
# =========================================================

import aiohttp
import asyncio
import json
import os
import random
from datetime import datetime

# === Archivos de entrada y salida ===
INPUT_FILE = "/home/ismael/Desktop/Diario_de_un_farmer/Plume_network/data/plume_wallets.json"
OUTPUT_FILE = "/home/ismael/Desktop/Diario_de_un_farmer/Plume_network/data/plume_wallets_enriched.json"

# === Endpoints de API ===
PLUME_STATS_API = "https://portal-api.plume.org/api/v1/stats/wallet"
PLUME_EXPLORER_API = "https://explorer.plume.org/api"

# === Configuración de concurrencia ===
MAX_CONCURRENT_REQUESTS = 300
BATCH_SIZE = 500   # Guardar progreso cada 500 wallets
RETRY_LIMIT = 5    # Reintentos en caso de error
RETRY_BACKOFF = [1, 2, 5, 10]  # tiempo de espera creciente


# =======================
# Funciones de utilidad
# =======================

async def fetch_json(session, url, params=None, retries=0):
    """Realiza request GET con reintentos y backoff exponencial."""
    try:
        async with session.get(url, params=params, timeout=20) as response:
            if response.status != 200:
                raise Exception(f"HTTP {response.status}")
            return await response.json()
    except Exception as e:
        if retries < RETRY_LIMIT:
            wait_time = RETRY_BACKOFF[min(retries, len(RETRY_BACKOFF)-1)]
            print(f"⚠️ Error fetch_json {url}: {e}. Reintentando en {wait_time}s...")
            await asyncio.sleep(wait_time + random.random())
            return await fetch_json(session, url, params, retries + 1)
        else:
            print(f"❌ Error permanente {url}: {e}")
            return None


async def fetch_wallet_stats(session, wallet):
    """Consulta stats de una wallet en la API de Plume."""
    url = f"{PLUME_STATS_API}?walletAddress={wallet}"
    data = await fetch_json(session, url)
    if not data:
        return None

    stats = data.get("data", {}).get("stats", {})
    if not stats:
        return None

    referred_by_user = stats.get("referredByUser")
    if referred_by_user and isinstance(referred_by_user, dict):
        referred_by_user = referred_by_user.get("walletAddress")
    else:
        referred_by_user = None

    return {
        "walletAddress": stats.get("walletAddress"),
        "totalXp": stats.get("totalXp", 0),
        "referrals": stats.get("referrals", 0),
        "referredByUser": referred_by_user,
        "protectorOfPlumePoints": stats.get("protectorsOfPlumePoints", 0)
    }


async def fetch_transactions(session, wallet):
    """Consulta transacciones de una wallet en el Explorer."""
    params = {
        'module': 'account',
        'action': 'txlist',
        'address': wallet
    }
    data = await fetch_json(session, PLUME_EXPLORER_API, params=params)
    if data and data.get("message") == "OK":
        return data.get("result", [])
    return []


async def get_activity_stats(session, wallet):
    """Calcula número de transacciones y días activos de una wallet."""
    transactions = await fetch_transactions(session, wallet)
    if not transactions:
        return {"txn": 0, "activeDays": 0}

    daily_counts = {}
    for tx in transactions:
        try:
            tx_date = datetime.fromtimestamp(int(tx['timeStamp'])).date()
            daily_counts[tx_date] = daily_counts.get(tx_date, 0) + 1
        except:
            continue

    return {
        "txn": len(transactions),
        "activeDays": len(daily_counts)
    }


def evaluate_sybil(stats):
    """
    Clasifica una wallet como 'sybil', 'suspicious' o 'false'
    usando reglas heurísticas simples.
    """
    total_xp = stats.get("totalXp", 0) or 0
    protector = stats.get("protectorOfPlumePoints", 0) or 0
    txn = stats.get("txn", 0)
    active_days = stats.get("activeDays", 0)

    if total_xp == protector or (protector / total_xp > 0.8 if total_xp > 0 else False):
        return "true"
    if total_xp > 0 and 0.4 <= protector / total_xp <= 0.8:
        return "suspicious"
    if protector == 0 and txn == 0 and total_xp > 1000:
        return "true"
    if txn < 20 or active_days < 10:
        return "true"
    if total_xp > 0 and txn == 0:
        return "true"
    return "false"


# =======================
# Pipeline principal
# =======================

async def process_wallet(session, wallet, sem):
    """Procesa una wallet: stats + actividad + flag sybil."""
    async with sem:
        stats = await fetch_wallet_stats(session, wallet)
        if not stats:
            return None

        activity = await get_activity_stats(session, wallet)
        result = {**stats, **activity}
        result["sybilFlag"] = evaluate_sybil(result)
        return result


async def main():
    # Cargar wallets base
    with open(INPUT_FILE, "r") as f:
        wallets_data = json.load(f)
    wallets = [w["walletAddress"] for w in wallets_data]

    # Configuración de concurrencia
    sem = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)
    results = []

    async with aiohttp.ClientSession() as session:
        tasks = [process_wallet(session, w, sem) for w in wallets]
        for i, future in enumerate(asyncio.as_completed(tasks), start=1):
            res = await future
            if res:
                results.append(res)

            # Guardar progreso cada BATCH_SIZE wallets
            if i % BATCH_SIZE == 0:
                with open(OUTPUT_FILE, "w") as f:
                    json.dump(results, f, indent=2)
                print(f"💾 Progreso guardado: {i}/{len(wallets)} wallets procesadas...")

    # Guardar archivo final
    with open(OUTPUT_FILE, "w") as f:
        json.dump(results, f, indent=2)

    print(f"✅ Procesadas {len(results)} wallets. Archivo en {OUTPUT_FILE}")


if __name__ == "__main__":
    asyncio.run(main())


In [ ]:
# =========================================================
# Fase 3: Construcción de redes de referidos
#
# En esta fase construimos estructuras en forma de árbol,
# donde cada root es una wallet sin "referredByUser".
# 
# Cada nodo incluye:
#   - Datos de la wallet
#   - Sus referidos anidados en "referredWallets"
#   - El conteo de referidos directos e indirectos
#
# Guardamos las redes en un archivo JSON.
# =========================================================

import json

# === Archivos de entrada y salida ===
INPUT_FILE = "/home/ismael/Desktop/Diario_de_un_farmer/Plume_network/data/plume_wallets_enriched.json"
OUTPUT_FILE = "/home/ismael/Desktop/Diario_de_un_farmer/Plume_network/data/plume_networks.json"


# =======================
# Funciones
# =======================

def build_wallet_dict(wallets):
    """Crea un diccionario {walletAddress: wallet} inicializando listas y contadores."""
    wallet_dict = {}
    for w in wallets:
        wallet = w.copy()
        wallet["referredWallets"] = []  # lista de descendientes
        wallet["referralCount"] = 0     # se calculará después
        wallet_dict[wallet["walletAddress"]] = wallet
    return wallet_dict


def build_referral_tree(wallet_dict):
    """Construye los árboles de referidos a partir del diccionario de wallets."""
    roots = []

    for wallet in wallet_dict.values():
        parent_addr = wallet.get("referredByUser")
        if parent_addr and parent_addr in wallet_dict:
            # Insertamos esta wallet en la lista de referidos del padre
            wallet_dict[parent_addr]["referredWallets"].append(wallet)
        else:
            # Si no tiene un referidor válido → es root
            roots.append(wallet)

    return roots


def compute_referral_counts(wallet):
    """Cuenta recursivamente los referidos directos e indirectos de una wallet."""
    count = len(wallet["referredWallets"])
    for child in wallet["referredWallets"]:
        count += compute_referral_counts(child)
    wallet["referralCount"] = count
    return count


def build_networks(wallets):
    """Pipeline completo: crea el diccionario, arma los árboles y calcula los conteos."""
    wallet_dict = build_wallet_dict(wallets)
    roots = build_referral_tree(wallet_dict)

    for root in roots:
        compute_referral_counts(root)

    return roots


# =======================
# MAIN
# =======================

def main():
    # Leer archivo de wallets enriquecidas
    with open(INPUT_FILE, "r") as f:
        wallets = json.load(f)

    # Construir redes
    networks = build_networks(wallets)

    # Guardar en archivo JSON
    with open(OUTPUT_FILE, "w") as f:
        json.dump(networks, f, indent=2)

    print(f"✅ Redes construidas y guardadas en {OUTPUT_FILE}")
    print(f"🔹 Total redes: {len(networks)}")


if __name__ == "__main__":
    main()


In [ ]:
# ================================================
# Fase 4: Análisis y Conclusiones
# ================================================
# En esta fase realizamos un análisis global de las redes construidas.
# - Recorremos todas las wallets, incluyendo las anidadas dentro de referidos.
# - Calculamos estadísticas generales: total de wallets, sybils, legítimas, XP total.
# - Identificamos cuántas redes son dominadas por sybils (>80%).
# - Detectamos las redes más grandes y su peso en el ecosistema.
# - Extraemos algunos insights útiles sobre la estructura del ecosistema Plume.
# ================================================

import json

# =======================
# Configuración
# =======================
INPUT_FILE = "/home/ismael/Desktop/Diario_de_un_farmer/Plume_network/data/plume_networks.json"


# =======================
# Funciones
# =======================

def flatten_wallets(wallet, acc):
    """
    Recorre recursivamente un árbol de wallets y acumula todas en acc.
    """
    acc.append(wallet)
    for child in wallet.get("referredWallets", []):
        flatten_wallets(child, acc)
    return acc


def collect_all_wallets(networks):
    """
    Aplana todas las redes y devuelve una lista con TODAS las wallets.
    """
    all_wallets = []
    for net in networks:
        flatten_wallets(net, all_wallets)
    return all_wallets


def analyze_network(wallet):
    """
    Analiza una red individual y devuelve:
    - Número de wallets
    - XP total
    - % sybil
    """
    wallets = flatten_wallets(wallet, [])
    total = len(wallets)
    xp = sum(w.get("totalXp", 0) for w in wallets)
    sybil_count = sum(1 for w in wallets if w.get("sybilFlag") == "true")
    sybil_pct = (sybil_count / total * 100) if total > 0 else 0
    return {
        "rootWallet": wallet["walletAddress"],
        "walletCount": total,
        "totalXp": xp,
        "sybilPct": sybil_pct
    }


# =======================
# MAIN
# =======================

def main():
    # Leer archivo
    with open(INPUT_FILE, "r") as f:
        networks = json.load(f)

    # Aplanar todas las wallets
    all_wallets = collect_all_wallets(networks)

    # --- Stats globales ---
    total_wallets = len(all_wallets)
    sybil_wallets = sum(1 for w in all_wallets if w.get("sybilFlag") == "true")
    legit_wallets = total_wallets - sybil_wallets

    total_xp = sum(w.get("totalXp", 0) for w in all_wallets)
    xp_sybil = sum(w.get("totalXp", 0) for w in all_wallets if w.get("sybilFlag") == "true")
    xp_legit = total_xp - xp_sybil

    # --- Stats por red ---
    network_stats = [analyze_network(n) for n in networks]
    network_stats_sorted = sorted(network_stats, key=lambda x: x["walletCount"], reverse=True)

    # Redes sybilizadas (>80%)
    heavily_sybil_nets = [n for n in network_stats if n["sybilPct"] > 80]

    # --- Resultados ---
    print(" Fecha del análisis: 20/08/2025")
    print("=== Estadísticas Generales ===")
    print(f" Total de wallets: {total_wallets:,}")
    print(f"   ├─ Wallets Sybil: {sybil_wallets:,} ({sybil_wallets/total_wallets*100:.2f}%)")
    print(f"   └─ Wallets Legítimas: {legit_wallets:,} ({legit_wallets/total_wallets*100:.2f}%)")
    print()
    print(f" XP total acumulado: {total_xp:,}")
    print(f"   ├─ XP Sybil: {xp_sybil:,} ({xp_sybil/total_xp*100:.2f}%)")
    print(f"   └─ XP Legítimo: {xp_legit:,} ({xp_legit/total_xp*100:.2f}%)")
    print()
    print(" === Redes ===")
    print(f"🔹 Total de redes: {len(networks):,}")
    print(f"🔹 Redes con >80% sybil: {len(heavily_sybil_nets):,}")
    print()
    print("🏆 Top 3 redes más grandes (por número de wallets):")
    for i, net in enumerate(network_stats_sorted[:3], start=1):
        print(f"   {i}. Root: {net['rootWallet']}")
        print(f"      Wallets: {net['walletCount']:,}, XP: {net['totalXp']:,}, % Sybil: {net['sybilPct']:.2f}%")

    # --- Insights adicionales ---
    avg_wallets_per_net = total_wallets / len(networks) if networks else 0
    print()
    print(" Insights adicionales:")
    print(f"🔹 Promedio de wallets por red: {avg_wallets_per_net:.2f}")
    print(f"🔹 Red más grande contiene {network_stats_sorted[0]['walletCount']:,} wallets")
    print(f"🔹 Red más pequeña contiene {network_stats_sorted[-1]['walletCount']:,} wallets")


if __name__ == "__main__":
    main()


=== Estadísticas Generales ===
 Total de wallets: 247,761
   ├─ Wallets Sybil: 166,093 (67.04%)
   └─ Wallets Legítimas: 81,668 (32.96%)

 XP total acumulado: 1,444,431,850
   ├─ XP Sybil: 267,286,468 (18.50%)
   └─ XP Legítimo: 1,177,145,382 (81.50%)

 === Redes ===
🔹 Total de redes: 105,399
🔹 Redes con >80% sybil: 72,541

🏆 Top 3 redes más grandes (por número de wallets):
   1. Root: 0x87cb894a65be758f1624ccd9c6dc0716e97c5fec
      Wallets: 33,938, XP: 7,040,950, % Sybil: 100.00%
   2. Root: 0x53ab2038ad0a68b986cc43e5c4fa66457886dcd4
      Wallets: 9,816, XP: 8,737,271, % Sybil: 98.31%
   3. Root: 0x537e466b329f5f439059e69ad8d144637fd461d4
      Wallets: 5,027, XP: 26,209,243, % Sybil: 75.15%

 Insights adicionales:
🔹 Promedio de wallets por red: 2.35
🔹 Red más grande contiene 33,938 wallets
🔹 Red más pequeña contiene 1 wallets
